# Transfer learning, finetune VGG16

## 1. Why VGG16

VGG16 is a convolutional neural network model proposed by K. Simonyan and A. Zisserman from the University of Oxford in the paper "Very Deep Convolutional Networks for Large-Scale Image Recognition". It is one of the most popular pre-trained models for image classification tasks.

### Key Features of VGG16
1. Architecture: VGG16 is a convolutional neural network model proposed by K. Simonyan and A. Zisserman from the University of Oxford in the paper "Very Deep Convolutional Networks for Large-Scale Image Recognition".<br>
2. Layers: It has 16 layers with learnable weights, including 13 convolutional layers and 3 fully connected layers.<br>
3. Pre-trained Weights: The model is pre-trained on the ImageNet dataset, which contains 1.2 million images and 1000 classes.<br>
4. Usage: It is widely used for image classification tasks and can be fine-tuned for specific tasks such as brain tumor classification.

## 2. Finetune VGG16 for brain tumor classification task

### 2.1 Preprocessing data

In [1]:
import cv2
import os
import random
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from keras.utils import normalize
from PIL import Image
from sklearn.model_selection import train_test_split

In [2]:
seed = 99
tf.random.set_seed(seed)
np.random.seed(seed)
random.seed(seed)

#### Reading and nomalize dataset

In [3]:
no_dir = os.listdir('./data_no/data_no/NO/')
yes_dir = os.listdir('./data_yes/data_yes/YES/')
data_set,label = [],[]
for i,cur_img_dir in enumerate(no_dir):
    #check type of image
    if cur_img_dir.split('.')[1]=='jpg':
        img = cv2.imread('./data_no/data_no/NO/'+cur_img_dir)
        img = Image.fromarray(img,'RGB')
        img = img.resize((64,64))
        data_set.append(np.array(img))
        label.append(0)
for i,cur_img_dir in enumerate(yes_dir):
    #check type of image
    if cur_img_dir.split('.')[1]=='jpg':
        img = cv2.imread('./data_yes/data_yes/YES/'+cur_img_dir)
        img = Image.fromarray(img,'RGB')
        img = img.resize((64,64))
        data_set.append(np.array(img))
        label.append(1)

In [4]:
data_set = np.array(data_set)
label = np.array(label)

#### Splitting the data into training and testing

In [5]:
x_train,x_test,y_train,y_test = train_test_split(
    data_set,label,
    test_size=0.2,
    random_state=99
    )


### **read k fold cross validation**
### **using pytorch**
**run resnet**

In [6]:
print(f'X train shape: {x_train.shape}\nY train shape: {y_train.shape}\nX test shape: {x_test.shape}\nY test shape: {y_test.shape})')

X train shape: (873, 64, 64, 3)
Y train shape: (873,)
X test shape: (219, 64, 64, 3)
Y test shape: (219,))


#### Nomalize

In [7]:
x_train = normalize(x_train, axis=1)
x_test = normalize(x_test, axis=1)


### Loading pre-trained model

In [8]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [9]:
# Load the VGG16 model with pre-trained weights, excluding the top fully connected layers
base_model = VGG16(
    weights='imagenet', # bo hoac none
    include_top=False, 
    input_shape=(64, 64, 3)
    )

# Freeze the convolutional base
for layer in base_model.layers:
    layer.trainable = False # True for fine tune model


In [10]:
# Add custom top layers for classification
x = Flatten()(base_model.output)
x = Dense(512, activation='relu')(x)
x = Dense(1, activation='sigmoid')(x)  #binary classification 
# note 1 dense +1 activation

In [11]:
# Create the new model
model = Model(inputs=base_model.input, outputs=x)

In [12]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 64, 64, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 64, 64, 64)     │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 64, 64, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 32, 32, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 32, 32, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 16, 16, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 16, 16, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 16, 16, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 8, 8, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 8, 8, 512)      │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 8, 8, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 8, 8, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 4, 4, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 4, 4, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 4, 4, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 4, 4, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 2, 2, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     1,049,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           513 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15,764,289 (60.14 MB)

 Trainable params: 1,049,601 (4.00 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [14]:
model.compile(
    optimizer=Adam(learning_rate=1e-4, amsgrad=True),
    loss='binary_crossentropy', 
    metrics=['accuracy']
    )
# Train the model
history = model.fit(
    x_train, 
    y_train, 
    epochs=20, 
    batch_size=32, 
    validation_data = (x_test, y_test)
    )

Epoch 1/20
28/28 ━━━━━━━━━━━━━━━━━━━━ 23s 749ms/step - accuracy: 0.6138 - loss: 0.6539 - val_accuracy: 0.7808 - val_loss: 0.5039
Epoch 2/20
28/28 ━━━━━━━━━━━━━━━━━━━━ 19s 680ms/step - accuracy: 0.7949 - loss: 0.4884 - val_accuracy: 0.8037 - val_loss: 0.4575
Epoch 3/20
28/28 ━━━━━━━━━━━━━━━━━━━━ 18s 636ms/step - accuracy: 0.8121 - loss: 0.4495 - val_accuracy: 0.8219 - val_loss: 0.4351
Epoch 4/20
28/28 ━━━━━━━━━━━━━━━━━━━━ 18s 631ms/step - accuracy: 0.8244 - loss: 0.4260 - val_accuracy: 0.8447 - val_loss: 0.4171
Epoch 5/20
28/28 ━━━━━━━━━━━━━━━━━━━━ 18s 641ms/step - accuracy: 0.8304 - loss: 0.4072 - val_accuracy: 0.8539 - val_loss: 0.4022
Epoch 6/20
28/28 ━━━━━━━━━━━━━━━━━━━━ 19s 685ms/step - accuracy: 0.8458 - loss: 0.3913 - val_accuracy: 0.8493 - val_loss: 0.3891
Epoch 7/20
28/28 ━━━━━━━━━━━━━━━━━━━━ 17s 623ms/step - accuracy: 0.8570 - loss: 0.3773 - val_accuracy: 0.8493 - val_loss: 0.3775
Epoch 8/20
28/28 ━━━━━━━━━━━━━━━━━━━━ 17s 603ms/step - accuracy: 0.8628 - loss: 0.3645 - val_accu

**Infor**<br>
Name: BrainTurmor_v3<br>
Accuracy in test set: 0.9048<br>
Status: Good

In [15]:
model.save('BrainTurmor_v3.keras')